##推薦作法:
- 將需要的資料都存成變數：
    - 新聞內容：class(python裡class是保留字，所以我在下面的code改成category),title,newsDate,journalist,content,hit,url,source
    - 評論：url,commenter,comment
- 將這些變數用list或dictionary存起來
- 將存的list或dictionary轉成pandas
- 直接用DataFrame.to_csv(filename,encoding="utf-8")轉成csv
- 下面的code有存進資料庫的部分，如果不存資料庫的人可以把那些註解掉，存成csv就好，如果要存進資料庫，則記得先見好資料庫，資料庫的code詳見news.sql

## 抓取單一日期或日期區間  ETToday_day(startday,endday,sleep_time=0) 

In [13]:
# startday, endday 要輸入string(ex. '2015-1-1')
# 沒輸入日期預設抓取昨天新聞
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒

import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source=None):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
sentinel = object()
def ETToday_day(startday=(str(date.today()-timedelta(1))),endday=sentinel,sleep_time=0):
     
    if endday is sentinel:
        endday=startday
    
    def convert_day(d):
        return datetime.strptime(d, '%Y-%m-%d')
    
    ed=convert_day(endday)
    sd=convert_day(startday)

    assert ed >= sd, "endday must after startday"

    ds = []
    for i in xrange(0,(ed-sd).days+1):
        ds.append(str((ed- timedelta(i)).strftime('%Y-%m-%d')))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                #################連資料庫用
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        #################連資料庫用
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    if startday==endday:
        std="".join(startday.split('-'))
        filename="ETToday"+"_"+std+".csv"
    else:
        std="".join(startday.split('-'))
        end="".join(endday.split('-'))
        filename="ETToday"+"_"+std+"-"+end+".csv"
    print filename
    f=open(filename,"w")
    df.to_csv(filename,encoding="utf-8")
    f.close()


In [18]:
# run
ETToday_day('2015-06-04')

TypeError: must be string, not object

## 抓取單一或多月份  ETToday_month(year,month,year2,month2,sleep_time=0) 

In [14]:
# sleep_time意思為抓取每一天新聞之間要休息多久，單位為秒
# year, month要輸入integer(ex. 2012,3)
# 如果抓本月的新聞，會預設抓到今天前一天的新聞

import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import calendar

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source=None):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用

sentinel = object()
def ETToday_month(year,month,year2=sentinel,month2=sentinel,sleep_time=0):
    
    if year2 is sentinel:
        year2 = year
    if month2 is sentinel:
        month2 = month
    
    assert date(year2,month2,1)>=date(year,month,1), "year2,month2 must after year,month"

    def first_day_of_month_d(d):
        return date(d.year, d.month, 1)

    def first_day_of_month(y,m):
        return date(y,m,1)

    def last_day_of_month(y,m):
        return date(y,m+1,1) - timedelta(days = 1)

    def diff_months_d(d1, d2):
        return (d1.year - d2.year)*12 + d1.month - d2.month

    def diff_days(y,m,y2,m2):
        return abs(last_day_of_month(y2,m2)-first_day_of_month(y,m)).days

    # check if input month exceed current month
    assert diff_months_d(date.today(),first_day_of_month(year,month))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month
    assert diff_months_d(date.today(),first_day_of_month(year2,month2))>=0, "眼睛睜大點好嘛？%s月還沒到ok？"%month2

    if diff_months_d(date.today(),first_day_of_month(year2,month2))==0:
        duration=(date.today()-first_day_of_month_d(date.today())).days
        ds = []
        for i in xrange(1,duration+1):
            ds.append(str((date.today()- timedelta(i)).strftime('%Y-%m-%d')))
    else:
        #duration=calendar.monthrange(year,month)[1]
        duration=diff_days(year,month,year2,month2)
        ds = []
        for i in xrange(0,duration+1):
            ds.append(str((last_day_of_month(year2,month2)- timedelta(i)).strftime('%Y-%m-%d')))

    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                content = soup.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                #################連資料庫用
                connect("news","","localhost","root")
                insert_news()
                close()
                connect("comments","","localhost","root")
                insert_comments()
                close()
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        content = soup.select('.story > sectione > p')[1:]
                        for i in xrange(0,len(content)):
                            for con in content[i].select('img'):
                                content[i].img.decompose()
                        jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        journalist = (jour.split("記者")[1]).split("／")[0]
                        L.append({'Category':block.select('em')[0].text,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        #################連資料庫用
                        connect("news","","localhost","root")
                        insert_news()
                        close()
                        connect("comments","","localhost","root")
                        insert_comments()
                        close()
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    if (str(year)+str(month))==(str(year2)+str(month2)):
        filename="ETTdoay_%d%02d.csv"%(year,month)
    else:
        filename="ETTdoay_%d%02d-%d%02d.csv"%(year,month,year2,month2)
    f=open(filename,"w")
    df.to_csv(filename,encoding="utf-8")
    f.close()

In [ ]:
# run
ETToday_month(2015,5)

##直接輸入從今天算起要抓幾天 ETToday(days,sleep_time=0)

In [396]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time

#################連資料庫用
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()
#################連資料庫用
    
def ETToday(days,sleep_time=0):
    #day = str(date.today()- timedelta(1))
    
    hit = None
    ds = []
    for i in xrange(1,days+1):
        ds.append(str(date.today()- timedelta(i)))
    
    for day in ds:
        urls_dic = {"政治":"http://www.ettoday.net/news/news-list-%s-1.htm"%day,
                    "財經":"http://www.ettoday.net/news/news-list-%s-17.htm"%day,
                    "國際":"http://www.ettoday.net/news/news-list-%s-2.htm"%day,
                    "大陸":"http://www.ettoday.net/news/news-list-%s-3.htm"%day,
                    "社會":"http://www.ettoday.net/news/news-list-%s-6.htm"%day,
                    "地方":"http://www.ettoday.net/news/news-list-%s-7.htm"%day,
                    "影劇":"http://www.ettoday.net/news/news-list-%s-9.htm"%day,
                    "3C":"http://www.ettoday.net/news/news-list-%s-20.htm"%day
                   }

        L = []
        for url in urls_dic:
            res = requests.get(urls_dic[url])
            res.encoding="utf-8"
            soup = BeautifulSoup(res.text)
            page = re.search('\((\d+).\)',soup.select(".info")[0].text).group(1)
            pageCount = int(math.ceil(float(page)/30))
            for block in soup.select("#all-news-list > h3"):
                title=block.select('a')[0].text
                aLink=block.select('a')[0]['href']
                driver = webdriver.PhantomJS()
                driver.get(aLink)
                try:
                    WebDriverWait(driver, 10).until(
                            EC.visibility_of_element_located((By.CLASS_NAME, "story"))
                        )
                    html = driver.page_source
                    cont = str(html.encode('utf-8'))
                    soupCont = BeautifulSoup(cont)
                finally:
                    driver.quit()
                content = soupCont.select('.story > sectione > p')[1:]
                for i in xrange(0,len(content)):
                    for con in content[i].select('img'):
                        content[i].img.decompose()
                try:
                    journalist=re.search('<p>([^圖].*?)／.*?報導<',cont).group(1)
                except:
                    pass
                try:
                    journalist=re.search('>文／(.*?)<',cont).group(1)
                except:
                    pass
                category=block.select('em')[0].text
                #jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                #journalist = (jour.split("記者")[1]).split("／")[0]
                L.append({'Category':category,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                #################連資料庫用
                conn=connect("practice","oblivn0374","localhost","root")
                insert_news(conn)
                #insert_comments(conn)
                close(conn)
                #################連資料庫用
                time.sleep(sleep_time)
            
            if pageCount > 1:
                for i in xrange(2,pageCount+1):
                    url_1=str(urls_dic[url]).split(".htm")[0] + "-" + str(i) + ".htm"
                    res = requests.get(url_1)
                    res.encoding="utf-8"
                    soup = BeautifulSoup(res.text)
                    for block in soup.select("#all-news-list > h3"):
                        title=block.select('a')[0].text
                        aLink=block.select('a')[0]['href']
                        driver = webdriver.PhantomJS()
                        driver.get(aLink)
                        try:
                            WebDriverWait(driver, 10).until(
                                    EC.visibility_of_element_located((By.CLASS_NAME, "story"))
                                )
                            html = driver.page_source
                            cont = str(html.encode('utf-8'))
                            soupCont = BeautifulSoup(cont)
                        finally:
                            driver.quit()
                        content = soupCont.select('.story > sectione > p')[1:]
                        try:
                            for i in xrange(0,len(content)):
                                for con in content[i].select('img'):
                                    content[i].img.decompose()
                        except:
                            pass
                        try:
                            journalist=re.search('<p>([^圖].*?)／.*?報導<',cont).group(1)
                        except:
                            pass
                        try:
                            journalist=re.search('>文／(.*?)<',cont).group(1)
                        except:
                            pass
                        #jour = str(soup.select('.story > sectione > p')[0].text.encode('utf-8'))
                        #journalist = (jour.split("記者")[1]).split("／")[0]
                        category=block.select('em')[0].text
                        L.append({'Category':category,'Title':title,'Content':content,'Journalist':journalist,'Link':aLink,'Time':day,'Source':'ETToday'})
                        #################連資料庫用
                        conn=connect("practice","oblivn0374","localhost","root")
                        insert_news(conn,category,title,day,journalist,content,hit,alink)
                        #insert_comments(conn)
                        close(conn)
                        #################連資料庫用
                        time.sleep(sleep_time)
            
    # pandas
    import pandas as pd
    df = pd.DataFrame(L,columns =['Category','Title','Content','Journalist','Link','Time','Source'])

    # write in csv
    f=open("ETToday.csv","w")
    df.to_csv("ETToday",encoding="utf-8")
    f.close()


In [397]:
# run
ETToday(1)

TimeoutException: Message: 
Screenshot: available via screen


In [394]:
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=title,newsDate=day,journalist=journalist,content=content,hit=hit,url=aLink,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,url,commenter,comment,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(url,commenter,comment)values(%s,%s,%s)" ,[url,commenter,comment])
    db.commit()
def close(db):
    db.close()

NameError: name 'title' is not defined

In [389]:
url="aaaa"
commenter="abb"
comment="cc"
a=connect("practice","oblivn0374","localhost","root")
insert_comments(a,url,commenter,comment)
insert_news(a,"1","2","3","4","5","6","7","8")
close(a)
#connect("practice","oblivn0374","localhost","root")
